In [ ]:
import numpy as np
import tnn
import torch
import torch.nn as nn
import torch.utils.data as data
import logging

from datasets import load_dataset, concatenate_datasets
from transformers import RobertaTokenizer, RobertaForSequenceClassification

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict]
for logger in loggers:
    if "transformers" in logger.name.lower():
        logger.setLevel(logging.ERROR)

In [ ]:
dataset = load_dataset("stanfordnlp/imdb")
train = dataset.get("train")
test = dataset.get("test")

In [ ]:
num_labels = len(train.unique("label"))
name = "distilbert/distilroberta-base"
tokenizer = RobertaTokenizer.from_pretrained(name)

In [ ]:
def pre_process(examples):
    reviews = list(map(lambda review: review.lower(), examples["text"]))
    examples = tokenizer(
        reviews,
        max_length=512,
        truncation=True,
        padding="max_length",
    )
    return examples

train_dataset = train.map(
    pre_process, batched=True, num_proc=2,
)
test_dataset = test.map(
    pre_process, batched=True, num_proc=2,
)

In [ ]:
def collate_fn(batch):
    inputs = {"input_ids": [], "attention_mask": []}
    labels = []
    for example in batch:
        inputs["input_ids"].append(example["input_ids"])
        inputs["attention_mask"].append(example["attention_mask"])
        labels.append(example["label"])

    inputs["input_ids"] = torch.tensor(inputs["input_ids"]).long()
    inputs["attention_mask"] = torch.tensor(inputs["attention_mask"]).long()
    labels = torch.tensor(labels).long()
    return inputs, labels

In [ ]:
trainloader = data.DataLoader(
    train_dataset,
    batch_size=32,
    collate_fn=collate_fn,
    drop_last=False,
    shuffle=False,
    pin_memory=True,
    num_workers=2,
)

testloader = data.DataLoader(
    test_dataset,
    batch_size=32,
    collate_fn=collate_fn,
    drop_last=False,
    shuffle=False,
    pin_memory=True,
    num_workers=2,
)


def to(inputs, labels, device, non_blocking):
    inputs = dict(
        map(
            lambda item: (item[0], item[1].to(device, non_blocking=non_blocking)),
            inputs.items(),
        )
    )
    labels = labels.to(device, non_blocking=True)
    return inputs, labels

In [ ]:
class DistilRoberta(tnn.Model):

    def __init__(self, name, num_labels):
        super().__init__()
        self.distil_roberta = RobertaForSequenceClassification.from_pretrained(name, num_labels=num_labels)

    def forward(self, **inputs):
        return self.distil_roberta(**inputs)

In [ ]:
lr = 2e-5
weight_decay = 1e-2

model = DistilRoberta(name=name, num_labels=num_labels)
optim = torch.optim.SGD(model.parameters(), lr=lr, momentum=0, weight_decay=weight_decay)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
trainer = tnn.Trainer(
    model,
    optim,
    loss_fn,
    trainloader,
    testloader,
    epochs=10,
    unpack_inputs=True,
    save_weights=True,
    device=device,
    to_fn=to,
    path="../training/imdb-distil-roberta-base.h5",
    verbose=True,
    profile=True,
)

In [ ]:
base_metrics = trainer.train()